<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/S2S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference Source : wikidocs.net 유영준님 자료

스스로 학습하면서 필요한 부분에는 추가적 설명, 소스 코드 삽입 및 수정 등이 있습니다. 영리적 목적이 아닌, 자기 계발 목적으로 정리한 자료입니다.

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
#위키독스에 있는 대로 자료 받으면, 데이터 구성이 좀 달라서 변형시킴
lines = pd.read_csv("/gdrive/My Drive/RogerHeederer/NLP_entry/data/fra.txt", sep='\t')
lines.columns = ["src", "tar", "etc"]
lines.drop('etc', axis='columns', inplace=True)

In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]
lines.sample(10)

,src,tar
20818,She made me hurry.,Elle m'a fait courir.
37609,She is angry with me.,Elle est en colère après moi.
5671,I need a plan.,J'ai besoin d'un plan.
9062,She plays Bach.,Elle joue du Bach.
33914,Bring me the Kleenex.,Apporte-moi les Kleenex.
33967,Can you do it faster?,Arrives-tu à le faire plus rapidement ?
11559,I'm a volunteer.,Je suis volontaire.
36848,If only he had known!,Si seulement il avait su !
12277,Sharks eat fish.,Les requins mangent des poissons.
28288,Are you ready to go?,Es-tu prête à partir ?


In [5]:
#심볼 태그 추가 <sos> = \t 시작, <eos> = \n 종료
lines.tar = lines.tar.apply(lambda x : '\t' + x + ' \n')
lines.sample(10)

,src,tar
13164,What a blessing!,\tQuelle bénédiction ! \n
12894,Tom writes well.,\tTom écrit bien. \n
31187,My hair is too long.,\tJ'ai les cheveux trop longs. \n
30235,"I want to play, too.",\tJe veux également jouer. \n
27908,You won everything.,\tTu as tout gagné. \n
29376,I am short of money.,\tJ'ai un problème d'argent. \n
23701,He lit the candles.,\tIl a allumé les cierges. \n
21887,We ran out of gas.,\tNous fûmes à court d'essence. \n
35933,I prefer it that way.,\tJe le préfère ainsi. \n
11162,I love this one.,\tJ'adore celui-ci. \n


In [7]:
#글자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
  for char in line: # 해당 라인의 1글자씩 읽음
    src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [8]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print(src_vocab_size)
print(tar_vocab_size)
print(src_vocab)
print(tar_vocab)

79
106
{'w', 'f', 'D', 't', 'e', 'p', 'X', 'C', '9', 'E', '7', 'S', 'W', 'T', ',', "'", 'L', '&', '8', '’', 'l', 'n', 'j', '/', '?', '-', 'o', 'c', 'y', 'm', 'M', 'u', '2', '!', 'F', 'G', 'O', 'R', 'I', 'V', 'v', 'P', '5', '0', 'A', 'a', '4', '1', 'd', ' ', 'k', '$', 'Y', '6', 'é', 's', 'h', '%', 'g', '"', 'H', 'Q', '.', 'U', 'r', 'K', 'z', 'Z', 'J', 'x', ':', '3', 'i', 'B', '€', 'N', 'b', 'q'}
{'\u200b', 'ï', 'C', 'ê', 'W', 'L', 'n', 'j', '«', 'œ', '2', 'Ê', 'P', 'v', '4', 'd', '‘', 's', 'H', 'U', 'Ô', 'î', 'É', ':', 'i', '»', 'b', 'D', 'p', 'S', 'û', '&', 'ç', '?', 'o', 'c', 'y', 'M', 'u', 'F', 'I', 'ë', '6', '\n', 'h', '\u2009', 'K', 'z', ')', '\u202f', 'f', 't', '9', 'ù', ',', "'", 'l', '’', 'm', '!', 'R', 'À', '0', 'A', ' ', '$', '\xa0', 'Y', 'С', '"', 'g', '.', 'J', 'x', '3', 'N', 'q', 'w', 'e', 'X', 'E', '7', 'T', '8', '\t', '(', '-', 'G', 'O', 'V', '5', 'è', 'a', '1', 'k', 'é', 'â', '%', 'Q', 'à', 'r', 'ô', 'B', 'Ç', 'Z'}


In [9]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [10]:
#글자 집합에 글자 단위로 저장은 되었고ㅡ 이제 인덱스를 부여해본다
src_to_index = dict( [(word, i+1) for i, word in enumerate(src_vocab)] )
tar_to_index = dict( [(word, i+1) for i, word in enumerate(tar_vocab)] )
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

In [11]:
# 영어 정수 인코딩 수행
encoder_input = []
for line in lines.src: #입력 데이터 문장에서 1 line씩 읽음
  temp_X = []
  for w in line: # 각 줄에서 1개 글자씩 읽음
    temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 반환
  encoder_input.append(temp_X)
print(encoder_input[:5])

list(lines.src[:5])

[[31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2], [46, 57, 64, 23]]


['Hi.', 'Hi.', 'Run!', 'Run!', 'Who?']

In [12]:
# 불어 정수 인코딩 수행
decoder_input = []
for line in lines.tar:
  temp_X = []
  for w in line:
    temp_X.append(tar_to_index[w])
  decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 45, 53, 64, 73, 72, 14, 3, 2], [1, 29, 67, 73, 70, 71, 105, 4, 3, 2], [1, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2], [1, 43, 73, 61, 3, 26, 3, 2]]


In [13]:
list(lines.tar[:5])

['\tSalut ! \n',
 '\tSalut. \n',
 '\tCours\u202f! \n',
 '\tCourez\u202f! \n',
 '\tQui ? \n']

In [14]:
# 프랑스어 정답의 맨 앞에 붙은 \t를 제거
decoder_target = []
for line in lines.tar:
  t = 0
  temp_X = []
  for w in line:
    if t>0: # \t 인덱스가 0이니 0은 무조건 스킵하는 구현
      temp_X.append(tar_to_index[w])
    t=t+1
  decoder_target.append(temp_X)
print(decoder_target[:5])

[[45, 53, 64, 73, 72, 3, 4, 3, 2], [45, 53, 64, 73, 72, 14, 3, 2], [29, 67, 73, 70, 71, 105, 4, 3, 2], [29, 67, 73, 70, 57, 78, 105, 4, 3, 2], [43, 73, 61, 3, 26, 3, 2]]


In [15]:
# 문장 패딩하기
# 영어와 프랑스어의 길이는 하나의 쌍이라도, 언어별 글자 수가 다르기 때문에 같은 길이로 패딩할 필요 없다.
# 영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이 맞춰서 패딩
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

24
75


In [16]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [17]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(60000, 24)
(60000, 75)
(60000, 75)


In [18]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

##여기까지가 전처리 

In [19]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [20]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [21]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [22]:

model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
750/750 [==============================] - 32s 43ms/step - loss: 0.7858 - val_loss: 0.6899
Epoch 2/50
750/750 [==============================] - 31s 42ms/step - loss: 0.4807 - val_loss: 0.5604
Epoch 3/50
750/750 [==============================] - 31s 42ms/step - loss: 0.4028 - val_loss: 0.4892
Epoch 4/50
750/750 [==============================] - 31s 41ms/step - loss: 0.3575 - val_loss: 0.4503
Epoch 5/50
750/750 [==============================] - 31s 41ms/step - loss: 0.3271 - val_loss: 0.4245
Epoch 6/50
750/750 [==============================] - 31s 42ms/step - loss: 0.3050 - val_loss: 0.4071
Epoch 7/50
750/750 [==============================] - 31s 42ms/step - loss: 0.2882 - val_loss: 0.3934
Epoch 8/50
750/750 [==============================] - 31s 42ms/step - loss: 0.2745 - val_loss: 0.3845
Epoch 9/50
750/750 [==============================] - 31s 42ms/step - loss: 0.2632 - val_loss: 0.3780
Epoch 10/50
750/750 [==============================] - 31s 42ms/step - loss: 0.253

KeyboardInterrupt: ignored

In [ ]:
#인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

#디코더 정의
decoder_state_input_h = Input(shape=(256,)) #이전 시점의 상태들을 저장한 텐서
decoder_state_input_c = Input(shape=(256,)) #이전 시점의 상태들을 저장한 텐서
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)